In [15]:
import os
from os import listdir
from os.path import isfile, join
import cv2
import numpy as np
import keras
from keras import backend as k
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers.core import Flatten, Dense

In [16]:
def resize(img):
    img = cv2.resize(img,(20,20))
    return img

In [17]:
X_train = []
y_train = []

# to get the name of the folder
for name_folder in os.listdir("C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\extracted_letter_images") :

    name = 'C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\extracted_letter_images/' + name_folder
    for f in listdir(name):
        # name of the folder is the name of the output
        y_train.append(np.asarray(name_folder))
        
        # constructing full path to the image
        name = 'C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\extracted_letter_images/' + name_folder + '/' + f
        
        # reading the image
        image = cv2.imread(name,0)/255
        
        # appending to form the image list
        image = np.asarray(image)
        image = resize(image)
        X_train.append([image])

In [18]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_train.shape

(38744, 1, 20, 20)

In [19]:
X_train = np.reshape(X_train, [-1,20,20,1])

In [20]:
X_train.shape

(38744, 20, 20, 1)

In [21]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(y_train)

onehot_encoder = OneHotEncoder(sparse=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
y_train = onehot_encoder.fit_transform(integer_encoded)

In [22]:
model = Sequential()

# First convolutional layer with max pooling
model.add(Conv2D(64, (3, 3), padding="same", input_shape=(20,20,1), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Second convolutional layer with max pooling
model.add(Conv2D(128, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# third convolutional layer with max pooling
model.add(Conv2D(512, (3, 3), padding="same", activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Hidden layer with 500 nodes
model.add(Flatten())
model.add(Dense(100, activation="relu"))

# Output layer with 32 nodes (one for each possible letter/number we predict)
model.add(Dense(32, activation="softmax"))

# Ask Keras to build the TensorFlow model behind the scenes
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 20, 20, 64)        640       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 64)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 10, 10, 128)       73856     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 5, 5, 128)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 5, 5, 512)         590336    
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 2, 2, 512)         0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 2048)             

In [24]:
X_train, X_test, y_train, y_test = train_test_split(X_train,y_train ,test_size = .2)

In [25]:
y_train.shape

(30995, 32)

In [ ]:
# fitting the model
model.fit(X_train, y_train,
          validation_data=[X_test,y_test],
          batch_size=2000, 
          epochs=10, 
          verbose=1)

Train on 30995 samples, validate on 7749 samples
Epoch 1/10
30995/30995 [==============================] - 116s 4ms/step - loss: 3.2901 - accuracy: 0.1437 - val_loss: 2.6890 - val_accuracy: 0.4180
Epoch 2/10
30995/30995 [==============================] - 116s 4ms/step - loss: 1.5717 - accuracy: 0.6410 - val_loss: 0.5808 - val_accuracy: 0.8537
Epoch 3/10
30995/30995 [==============================] - 114s 4ms/step - loss: 0.3224 - accuracy: 0.9267 - val_loss: 0.1503 - val_accuracy: 0.9742
Epoch 4/10
30995/30995 [==============================] - 116s 4ms/step - loss: 0.1057 - accuracy: 0.9823 - val_loss: 0.0753 - val_accuracy: 0.9884
Epoch 5/10
30995/30995 [==============================] - 116s 4ms/step - loss: 0.0582 - accuracy: 0.9911 - val_loss: 0.0528 - val_accuracy: 0.9910
Epoch 6/10
30995/30995 [==============================] - 117s 4ms/step - loss: 0.0413 - accuracy: 0.9925 - val_loss: 0.0428 - val_accuracy: 0.9924
Epoch 7/10
30995/30995 [==============================] - 123s 

In [ ]:
#saving the model 
model.save('C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\models\model1.h5')

In [ ]:
np.save('C:\\Users\AUTOBOTS\Downloads\optical-character-recognition-OCR-master\models/xtrain',X_train,allow_pickle=True)